In [ ]:
import cirq
import numpy as np

In [ ]:
def H_p(C, gamma, N):
    HCircuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(N)
    for i in range(0, N, 1):
        for j in range(i+1, i + N, 1):
            HCircuit.append(cirq.CNOT(qubits[i], qubits[j % N]))
            HCircuit.append(cirq.Z(qubits[j % N]) ** (gamma * C[i][j % N] / np.pi))
            HCircuit.append(cirq.CNOT(qubits[i], qubits[j % N]))
    return HCircuit

def H_d(N):
    dCircuit = cirq.Circuit()
    qubits = cirq.LineQubit.range(N)
    for i in range(N):
        dCircuit.append(cirq.X(qubits[i]))
    return dCircuit